In [1]:
import numpy as np

import scipy as sp
from scipy import spatial

import matplotlib.pyplot as plt

from nltk.stem.lancaster import LancasterStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import pairwise
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
import heapq
import string
import re

from nltk.corpus import stopwords
from collections import defaultdict


vectorizer = CountVectorizer()
st = LancasterStemmer()

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open("test/train.dat", "r") as fh:
    linesOfTrainData = fh.readlines()
print("Line count in training :" ,len(linesOfTrainData))

#with open("TestData/format.dat", "r") as fh:
with open("test/format.dat", "r") as fh:
    linesOfFormat = fh.readlines()
print("Line count in format :" ,len(linesOfFormat))

with open("test/test.dat", "r") as fh:
    linesOfTestData = fh.readlines()
print("Line count in test :" ,len(linesOfTestData))


Line count in training : 800
Line count in format : 350
Line count in test : 350


In [3]:
 #vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 50, stop_words = 'english', vocabulary=features)
  
vectorizer = CountVectorizer(analyzer='word', lowercase = True)

In [6]:

linesOfTrainData_Transformed = vectorizer.fit_transform(linesOfTrainData)
print(linesOfTrainData_Transformed[:1])

  (0, 88110)	1
  (0, 87805)	1
  (0, 87796)	1
  (0, 87697)	1
  (0, 87478)	1
  (0, 87451)	1
  (0, 87389)	1
  (0, 87316)	1
  (0, 87250)	1
  (0, 87091)	1
  (0, 86964)	1
  (0, 86787)	1
  (0, 86714)	1
  (0, 86673)	1
  (0, 86537)	1
  (0, 86414)	1
  (0, 86369)	1
  (0, 86352)	1
  (0, 86331)	1
  (0, 86199)	1
  (0, 86114)	1
  (0, 86082)	1
  (0, 85352)	1
  (0, 85208)	1
  (0, 85144)	1
  :	:
  (0, 32920)	1
  (0, 31940)	1
  (0, 31138)	1
  (0, 27601)	1
  (0, 25102)	1
  (0, 25022)	1
  (0, 22607)	1
  (0, 21017)	1
  (0, 18974)	1
  (0, 18122)	1
  (0, 17823)	1
  (0, 15745)	1
  (0, 15657)	1
  (0, 15365)	1
  (0, 12518)	1
  (0, 7127)	1
  (0, 4619)	1
  (0, 3044)	1
  (0, 2162)	1
  (0, 1994)	1
  (0, 59823)	1
  (0, 52283)	1
  (0, 50318)	1
  (0, 26104)	1
  (0, 8901)	1


In [8]:

linesOfTestData_Transformed = vectorizer.transform(linesOfTestData)
print(linesOfTestData_Transformed[:1])


  (0, 85)	1
  (0, 95)	1
  (0, 160)	1
  (0, 178)	1
  (0, 193)	1
  (0, 242)	1
  (0, 319)	1
  (0, 655)	1
  (0, 1096)	1
  (0, 1144)	1
  (0, 1225)	1
  (0, 1303)	1
  (0, 1379)	1
  (0, 1429)	1
  (0, 1533)	1
  (0, 1595)	1
  (0, 1713)	1
  (0, 1726)	1
  (0, 1819)	1
  (0, 1849)	1
  (0, 2010)	1
  (0, 2144)	1
  (0, 2286)	1
  (0, 2344)	1
  (0, 2565)	1
  :	:
  (0, 85596)	1
  (0, 85699)	1
  (0, 85774)	1
  (0, 86341)	1
  (0, 86368)	1
  (0, 86558)	1
  (0, 86573)	1
  (0, 86696)	1
  (0, 86813)	1
  (0, 86844)	1
  (0, 86868)	1
  (0, 86978)	1
  (0, 87043)	1
  (0, 87087)	1
  (0, 87273)	1
  (0, 87436)	1
  (0, 87511)	1
  (0, 87689)	1
  (0, 87695)	1
  (0, 87731)	1
  (0, 87773)	1
  (0, 87881)	1
  (0, 87937)	1
  (0, 87966)	1
  (0, 87974)	1


In [9]:
feature_names = vectorizer.get_feature_names() 
len(feature_names)


88112

In [10]:
def CalculateCosine(vt,vs):
        cosineSimilarityValue = cosine_similarity(vt,vs)
        return cosineSimilarityValue

In [11]:
cosineSimilarityValue = CalculateCosine(linesOfTestData_Transformed,linesOfTrainData_Transformed)


In [17]:

f = open('test/testData/format.dat', 'w')
count = 0
for row in cosineSimilarityValue:

    
    #kneighbours = heapq.nlargest(5, row)
    k=42
    partitioned_row_byindex = np.argpartition(-row, k)  
    similar_index = partitioned_row_byindex[:k]
    
    #print(similar_index)
    
    neighbourReviewTypeList = []
    neighbourReviewTypeNegative = 0
    neighbourReviewTypePositive = 0

    for index in similar_index:

        if linesOfTrainData[index].strip()[0] == '0':
            #neighbourReviewTypeList.append("-1")
            neighbourReviewTypeNegative+=1
        elif linesOfTrainData[index].strip()[0] == '1':
            #neighbourReviewTypeList.append("+1")
            neighbourReviewTypePositive+=1
            
    
    if neighbourReviewTypeNegative > neighbourReviewTypePositive:
        f.write('0\n')
        count+=1
    else:
        f.write('1\n')
        count+=1

print("count : ",count)
print("--The End--")

count :  350
--The End--
